In [858]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

df = pd.read_csv('Phenotypic_V1_0b_preprocessed1.csv')
print(df.shape)
df.head(5)

(1112, 13)


,Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,anat_cnr,anat_efc,anat_fber,anat_fwhm,anat_qi1,anat_snr
0,1,50002,1,50002,PITT,no_filename,1,10.201539,1.194664,16.223458,3.878000,0.152711,12.072452
1,2,50003,2,50003,PITT,Pitt_0050003,1,7.165701,1.126752,10.460008,4.282238,0.161716,9.241155
2,3,50004,3,50004,PITT,Pitt_0050004,1,7.698144,1.226218,9.725750,3.881684,0.174186,9.323463
3,4,50005,4,50005,PITT,Pitt_0050005,1,9.071807,1.256278,11.198226,3.628667,0.119269,10.814200
4,5,50006,5,50006,PITT,Pitt_0050006,1,8.026798,1.407166,6.282055,3.674539,0.130647,10.123574


In [859]:
df['DX_GROUP'] = df['DX_GROUP'].astype('category')
encode_map = {
    1: 0,
    2:1
}

df['DX_GROUP'].replace(encode_map, inplace=True)
df.head(5)

,Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,anat_cnr,anat_efc,anat_fber,anat_fwhm,anat_qi1,anat_snr
0,1,50002,1,50002,PITT,no_filename,0,10.201539,1.194664,16.223458,3.878000,0.152711,12.072452
1,2,50003,2,50003,PITT,Pitt_0050003,0,7.165701,1.126752,10.460008,4.282238,0.161716,9.241155
2,3,50004,3,50004,PITT,Pitt_0050004,0,7.698144,1.226218,9.725750,3.881684,0.174186,9.323463
3,4,50005,4,50005,PITT,Pitt_0050005,0,9.071807,1.256278,11.198226,3.628667,0.119269,10.814200
4,5,50006,5,50006,PITT,Pitt_0050006,0,8.026798,1.407166,6.282055,3.674539,0.130647,10.123574


In [860]:
#Dropping empty columns
df['DX_GROUP'].replace(2, 0, inplace=True) #So sigmoid function gives right output. if you replace sigmoid, you can skip this

df['anat_cnr'].replace('', np.nan, inplace=True)
df['anat_efc'].replace('', np.nan, inplace=True)
df['anat_fber'].replace('', np.nan, inplace=True)
df['anat_fwhm'].replace('', np.nan, inplace=True)
df['anat_qi1'].replace('', np.nan, inplace=True)
df['anat_snr'].replace('', np.nan, inplace=True)

#Replacing null values in all relevant input columns
df.dropna(subset=['anat_cnr','anat_efc', 'anat_fber', 'anat_fwhm', 'anat_qi1', 'anat_snr'], inplace=True)

#Verifying number of null rows
print("Number of null values:")
print(df.isnull().sum())

Number of null values:
Unnamed: 0    0
SUB_ID        0
X             0
subject       0
SITE_ID       0
FILE_ID       0
DX_GROUP      0
anat_cnr      0
anat_efc      0
anat_fber     0
anat_fwhm     0
anat_qi1      0
anat_snr      0
dtype: int64


In [861]:
X=df[['anat_cnr','anat_efc', 'anat_fber', 'anat_fwhm', 'anat_qi1', 'anat_snr']]
y=df['DX_GROUP']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69)

In [862]:
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.001

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#y_train =  torch.from_numpy(y_train.ravel()).float()
y_train = torch.Tensor(y_train.values)
y_test = torch.Tensor(y_test.values)

print(y_train)
print(X_test)

tensor([0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0.,
        0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0.,
        0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
        1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.,
        1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0.,
        0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0.,
        1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0.,
        1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
        1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
        0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
        0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
        0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0.,
        1., 1., 1., 0., 0., 0., 0., 0., 

In [863]:
#Function to train data
class TrainData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

#Calling dataset class to train on X and y tensors
train_data = TrainData(torch.FloatTensor(X_train), 
                       torch.FloatTensor(y_train))
#Function to test data
class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    
#Calling dataset class to test on X tensor
test_data = TestData(torch.FloatTensor(X_test))

In [864]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

In [865]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        self.layer_1 = nn.Linear(6, 3) 
        self.layer_2 = nn.Linear(3, 2)
        self.layer_out = nn.Linear(2, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(3)
        self.batchnorm2 = nn.BatchNorm1d(2)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [866]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = BinaryClassification()
model.to(device)
print(model)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


cpu
BinaryClassification(
  (layer_1): Linear(in_features=6, out_features=3, bias=True)
  (layer_2): Linear(in_features=3, out_features=2, bias=True)
  (layer_out): Linear(in_features=2, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [867]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [868]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Train Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Train Loss: 0.70427 | Acc: 51.833
Epoch 002: | Train Loss: 0.70235 | Acc: 51.917
Epoch 003: | Train Loss: 0.69682 | Acc: 53.167
Epoch 004: | Train Loss: 0.69392 | Acc: 54.083
Epoch 005: | Train Loss: 0.69537 | Acc: 53.667
Epoch 006: | Train Loss: 0.69546 | Acc: 54.000
Epoch 007: | Train Loss: 0.69215 | Acc: 54.833
Epoch 008: | Train Loss: 0.69029 | Acc: 54.083
Epoch 009: | Train Loss: 0.69636 | Acc: 52.750
Epoch 010: | Train Loss: 0.69042 | Acc: 54.583
Epoch 011: | Train Loss: 0.68872 | Acc: 53.333
Epoch 012: | Train Loss: 0.69138 | Acc: 54.833
Epoch 013: | Train Loss: 0.68821 | Acc: 54.250
Epoch 014: | Train Loss: 0.68997 | Acc: 53.500
Epoch 015: | Train Loss: 0.68914 | Acc: 54.333
Epoch 016: | Train Loss: 0.68942 | Acc: 54.500
Epoch 017: | Train Loss: 0.68989 | Acc: 53.750
Epoch 018: | Train Loss: 0.68746 | Acc: 52.583
Epoch 019: | Train Loss: 0.68738 | Acc: 55.000
Epoch 020: | Train Loss: 0.68864 | Acc: 53.583
Epoch 021: | Train Loss: 0.68784 | Acc: 56.500
Epoch 022: | 

In [869]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        
    y_pred_list = [a.squeeze().tolist() for a in y_pred_list]
#print(y_pred_list)    

In [ ]:
model.eval()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Valid Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

Epoch 001: | Valid Loss: 0.68370 | Acc: 55.083
Epoch 002: | Valid Loss: 0.68386 | Acc: 55.583
Epoch 003: | Valid Loss: 0.68454 | Acc: 55.250
Epoch 004: | Valid Loss: 0.68289 | Acc: 55.583
Epoch 005: | Valid Loss: 0.68190 | Acc: 55.667
Epoch 006: | Valid Loss: 0.68239 | Acc: 55.750
Epoch 007: | Valid Loss: 0.68260 | Acc: 55.250
Epoch 008: | Valid Loss: 0.68364 | Acc: 55.417
Epoch 009: | Valid Loss: 0.68373 | Acc: 55.250
Epoch 010: | Valid Loss: 0.68321 | Acc: 55.083
Epoch 011: | Valid Loss: 0.68268 | Acc: 55.250
Epoch 012: | Valid Loss: 0.68447 | Acc: 54.500
Epoch 013: | Valid Loss: 0.68315 | Acc: 54.250
Epoch 014: | Valid Loss: 0.68464 | Acc: 54.083
Epoch 015: | Valid Loss: 0.68235 | Acc: 54.250
Epoch 016: | Valid Loss: 0.68219 | Acc: 54.750
Epoch 017: | Valid Loss: 0.68208 | Acc: 54.333
Epoch 018: | Valid Loss: 0.68024 | Acc: 55.083
Epoch 019: | Valid Loss: 0.68155 | Acc: 53.583
Epoch 020: | Valid Loss: 0.68353 | Acc: 53.833
Epoch 021: | Valid Loss: 0.68250 | Acc: 53.917
Epoch 022: | 

In [ ]:
print(classification_report(y_test, y_pred_list))